## Import Library

In [82]:
import json
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel

#Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this 
import matplotlib.pyplot as plt
%matplotlib inline

## Load Data 

In [83]:
input_file ='../data/results_3k_2hop.json'

# with open(input_file) as f:
#     repo_data = json.load(f,strict=False)
#  JSONDecodeError: Extra data: line 2 column 1 (char 969)
    
file = open(input_file, 'r', encoding='utf-8')
repo_data = {}
for line in file.readlines():
    dic = json.loads(line)
    repo_data = {**repo_data,**dic}
    
print(f'Repository numbers: {len(repo_data)}')
print(f'''https://github.com/natcap/natgeo-dams' : 
{repo_data['https://github.com/natcap/natgeo-dams']}
{len(repo_data['https://github.com/natcap/natgeo-dams'])}''')

Repository numbers: 2960
https://github.com/natcap/natgeo-dams' : 
['pypng', 'requests', 'alabaster', 'codecov', 'detox', 'docutils', 'flake8', 'httpbin', 'more-itertools', 'pysocks', 'pytest', 'pytest-cov', 'pytest-httpbin', 'pytest-mock', 'pytest-xdist', 'readme-renderer', 'sphinx', 'tox', 'apipkg', 'appdirs', 'atomicwrites', 'attrs', 'babel', 'bleach', 'blinker', 'brotlipy', 'certifi', 'cffi', 'chardet', 'click', 'configparser', 'contextlib2', 'coverage', 'decorator', 'distlib', 'dnspython', 'entrypoints', 'enum34', 'eventlet', 'execnet', 'filelock', 'flask', 'funcsigs', 'functools32', 'greenlet', 'idna', 'imagesize', 'importlib-metadata', 'importlib-resources', 'itsdangerous', 'jinja2', 'markupsafe', 'mccabe', 'mock', 'monotonic', 'pathlib2', 'pluggy', 'py', 'pycodestyle', 'pycparser', 'pyflakes', 'pygments', 'pytest-forked', 'pytz', 'raven', 'scandir', 'singledispatch', 'six', 'snowballstemmer', 'toml', 'typing', 'urllib3', 'virtualenv', 'webencodings', 'werkzeug', 'zipp']
76


In [84]:
kw_file = '../data/keywords_description.json'

f = open(kw_file,encoding='utf-8')
kw_dict = json.load(f)
f.close()

In [85]:
# get the key words for each repository
keywords_dict = {}
for i in kw_dict['results']['bindings']:
    # 'https://github.com/' +
    repo_url_list =  i['soft']['value'].split('/')
    position = repo_url_list.index('Software')
    software_name = ''
    for j in range(position+1,len(repo_url_list)):
        software_name += repo_url_list[j] + '/'
    repo_url = 'https://github.com/' + software_name[:-1]
    
    key_words = i['description_keywords']['value']
    keywords_dict[repo_url] = key_words

len(keywords_dict)

12062

In [86]:
for repo_url in repo_data.keys():
    try:
        repo_data[repo_url].extend(keywords_dict[repo_url].split(','))
    except:
        pass
len(repo_kw_data) 

2783

In [87]:
print(f'Repository numbers: {len(repo_data)}')
print(f'''https://github.com/natcap/natgeo-dams' : 
{repo_data['https://github.com/natcap/natgeo-dams']}
{len(repo_data['https://github.com/natcap/natgeo-dams'])}''')

Repository numbers: 2960
https://github.com/natcap/natgeo-dams' : 
['pypng', 'requests', 'alabaster', 'codecov', 'detox', 'docutils', 'flake8', 'httpbin', 'more-itertools', 'pysocks', 'pytest', 'pytest-cov', 'pytest-httpbin', 'pytest-mock', 'pytest-xdist', 'readme-renderer', 'sphinx', 'tox', 'apipkg', 'appdirs', 'atomicwrites', 'attrs', 'babel', 'bleach', 'blinker', 'brotlipy', 'certifi', 'cffi', 'chardet', 'click', 'configparser', 'contextlib2', 'coverage', 'decorator', 'distlib', 'dnspython', 'entrypoints', 'enum34', 'eventlet', 'execnet', 'filelock', 'flask', 'funcsigs', 'functools32', 'greenlet', 'idna', 'imagesize', 'importlib-metadata', 'importlib-resources', 'itsdangerous', 'jinja2', 'markupsafe', 'mccabe', 'mock', 'monotonic', 'pathlib2', 'pluggy', 'py', 'pycodestyle', 'pycparser', 'pyflakes', 'pygments', 'pytest-forked', 'pytz', 'raven', 'scandir', 'singledispatch', 'six', 'snowballstemmer', 'toml', 'typing', 'urllib3', 'virtualenv', 'webencodings', 'werkzeug', 'zipp', 'repo',

## PreProcessing
1. Get distinct dependency file dictionary -> dep_dict
2. Sort dependency file  by frequency.     -> sort_dep


In [90]:
rep_list,dep_list = [],[]
for k,v in repo_data.items():
    rep_list.append(k)
    dep_list.append(v)
    
dep_dict = {}
for deps in repo_data.values():
    for i in deps:
        dep_dict[i] = dep_dict.get(i,0)+1

print(f'Distinct dependency file: {len(dep_dict)}',end='\n\n')
        
sort_dep = sorted(dep_dict.items(),key=lambda x: x[-1], reverse=True)
print(f'Frequent dependency files:  {sort_dep[:20]}')

Distinct dependency file: 43412

Frequent dependency files:  [('eslint', 18303), ('chalk', 16688), ('ansi-regex', 15700), ('ansi-styles', 15633), ('argparse', 15324), ('debug', 15126), ('ajv', 14907), ('glob', 14768), ('acorn', 14193), ('@babel/code-frame', 13530), ('@babel/highlight', 13387), ('anymatch', 13147), ('@types/node', 12956), ('ansi-escapes', 12614), ('rimraf', 12580), ('lodash', 12272), ('commander', 12212), ('arr-flatten', 12087), ('arr-diff', 12025), ('@babel/types', 11979)]


## Word Embedding
1. Build dependency file index table ->  id2word  format: {0: 'ipython',1: 'jupyter-sphinx',....}
2. Build Corpus   -> corpus    format: [[repo1 info],   [repo2 info]   ]  repo1_info:  [ (dep1_id:fre), xxx  ]
3. Build a LDA model

In [92]:
# based on dep file names , build dep name dictionary
id2word  = corpora.Dictionary(list(repo_data.values()))   # {0: 'emd-signal',1: 'numpy', 2: 'SQLAlchemy', 3: 'aiofiles' ....}

# based on dep name dict and dep names, build corpus
corpus = [id2word.doc2bow(text) for text in list(repo_data.values())] # [[(0, 1), (1, 1)],.....]


print(f'repos info: {corpus[0]}')    #(0, 1)

[[(id2word[id_], freq) for id_, freq in cp] for cp in corpus[:1]]

repos info: [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1)]


[[(' code', 1),
  (' dams', 1),
  (' grant', 1),
  (' natgeo', 1),
  ('alabaster', 1),
  ('apipkg', 1),
  ('appdirs', 1),
  ('atomicwrites', 1),
  ('attrs', 1),
  ('babel', 1),
  ('bleach', 1),
  ('blinker', 1),
  ('brotlipy', 1),
  ('certifi', 1),
  ('cffi', 1),
  ('chardet', 1),
  ('click', 1),
  ('codecov', 1),
  ('configparser', 1),
  ('contextlib2', 1),
  ('coverage', 1),
  ('decorator', 1),
  ('detox', 1),
  ('distlib', 1),
  ('dnspython', 1),
  ('docutils', 1),
  ('entrypoints', 1),
  ('enum34', 1),
  ('eventlet', 1),
  ('execnet', 1),
  ('filelock', 1),
  ('flake8', 1),
  ('flask', 1),
  ('funcsigs', 1),
  ('functools32', 1),
  ('greenlet', 1),
  ('httpbin', 1),
  ('idna', 1),
  ('imagesize', 1),
  ('importlib-metadata', 1),
  ('importlib-resources', 1),
  ('itsdangerous', 1),
  ('jinja2', 1),
  ('markupsafe', 1),
  ('mccabe', 1),
  ('mock', 1),
  ('monotonic', 1),
  ('more-itertools', 1),
  ('pathlib2', 1),
  ('pluggy', 1),
  ('py', 1),
  ('pycodestyle', 1),
  ('pycparser', 1)

In [93]:
%%time
clusters = list(range(2,30))
for k in clusters:   
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    # pprint(lda_model.print_topics())
    # Compute Perplexity and Coherence Score
    Perplexity = lda_model.log_perplexity(corpus)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=list(repo_data.values()), dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('cluster number:',k, 'Coherence Score',coherence_lda,'Perplexity:',Perplexity)
    print('='*55)

cluster number: 2 Coherence Score 0.6680153909417157 Perplexity: -7.515330049778937
cluster number: 3 Coherence Score 0.7565240370653714 Perplexity: -7.387527486745803
cluster number: 4 Coherence Score 0.8030349279018694 Perplexity: -7.35421383157777
cluster number: 5 Coherence Score 0.7378692645987809 Perplexity: -7.332937306083405
cluster number: 6 Coherence Score 0.7213421546539606 Perplexity: -7.330128431728066
cluster number: 7 Coherence Score 0.7216403494444983 Perplexity: -7.347507674964047
cluster number: 8 Coherence Score 0.692931712808801 Perplexity: -7.365108755501163
cluster number: 9 Coherence Score 0.6958168950563779 Perplexity: -7.411038985036263
cluster number: 10 Coherence Score 0.688689927890405 Perplexity: -7.478166970016604
cluster number: 11 Coherence Score 0.7346349897905575 Perplexity: -7.532862139589088
cluster number: 12 Coherence Score 0.6870571283367437 Perplexity: -7.618001847602874
cluster number: 13 Coherence Score 0.7742587187133879 Perplexity: -7.7074676


=> result: 
cluster number: 13 Coherence Score 0.7742587187133879 Perplexity: -7.707467603550034


In [94]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=13, 
                                       random_state=100,
                                       update_every=1,
                                       chunksize=100,
                                       passes=10,
                                       alpha='auto',
                                       per_word_topics=True)

In [95]:
# Show the top 5 words of each topic
for topic in lda_model.print_topics(num_words=5):
    print(topic)
    
# get the possible of each topic
lda_model.inference(corpus)[0].shape,lda_model.inference(corpus)[0]

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(vis, 'lda_clus=13.html')

(0, '0.006*"ts-jest" + 0.005*"graphql" + 0.005*"@types/jest" + 0.005*"codecov" + 0.005*"babel-preset-react"')
(1, '0.007*"webpack-cli" + 0.006*"ajv-errors" + 0.006*"webpack" + 0.006*"@webassemblyjs/wasm-opt" + 0.006*"@webassemblyjs/wasm-edit"')
(2, '0.005*"@babel/core" + 0.004*"eslint" + 0.004*"@types/node" + 0.004*"@babel/helper-plugin-utils" + 0.004*"@babel/helper-module-imports"')
(3, '0.003*"ansi-escapes" + 0.003*"acorn" + 0.003*"@babel/core" + 0.003*"chalk" + 0.003*"@babel/helpers"')
(4, '0.007*"mocha" + 0.005*"eslint" + 0.005*"chalk" + 0.005*"glob" + 0.004*"concat-map"')
(5, '0.003*"eslint" + 0.003*"@babel/core" + 0.003*"@babel/preset-env" + 0.003*"ansi-styles" + 0.003*"@babel/template"')
(6, '0.022*"pytest" + 0.021*"numpy" + 0.020*"sphinx" + 0.014*"matplotlib" + 0.014*"pytest-cov"')
(7, '0.004*"karma" + 0.003*"@types/color-name" + 0.003*"ansi-colors" + 0.003*"body-parser" + 0.003*"accepts"')
(8, '0.007*"babel-plugin-transform-es2015-modules-umd" + 0.007*"babel-helper-call-delega

C:\Users\Bin\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
